In [2]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow
import imageio
import imutils
import math
from scipy import ndimage
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from skimage import measure

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
os.chdir('./drive/My Drive/sudoku')

In [5]:
# This function finds the center of mass of an image and 
# calculates how much it needs to be shifted to be
# at the center 
def get_best_shift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

# Centralize an image according to its center of mass
# We will do the same thing with Sudoku Digit 
# so that every single digit image is in the SAME form
def shift_according_to_center_of_mass(img):
    img = cv2.bitwise_not(img)

    # Centralize the image according to center of mass
    shiftx,shifty = get_best_shift(img)
    shifted = shift(img,shiftx,shifty)
    img = shifted

    img = cv2.bitwise_not(img)
    return img

In [6]:
# Functions to take an unsolved sudoku puzzle and solve it

def solve(bo):
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if solve(bo):
                return True

            bo[row][col] = 0

    return False


def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True


def print_board(arr):
  final_result = []
  for i in range(9):
    row_result= []
    for j in range(9):
      row_result.append(arr[i][j])
    final_result.append(row_result)
  return np.array(final_result)


def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col

    return None



In [7]:
### Building a CNN model that was trained on the 
### Chars74K dataset so identify numbers

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
input_shape = (28, 28, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
model.load_weights('./digitRecognition.h5')

In [9]:
### This funciton takes each frame of the video and finds the
### sudoku grid in the image
def find_contour(original_image):
  orig_img_copy = original_image.copy()
  gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
  gaussian_blur = cv2.GaussianBlur(gray, (5,5), 0)
  thresh = cv2.adaptiveThreshold(gaussian_blur, 255, 1, 1, 11, 2)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
  max_contour = sorted(contours, key = cv2.contourArea, reverse=True)[0]
  epsilon = 0.01*cv2.arcLength(max_contour,True)
  approx = cv2.approxPolyDP(max_contour,epsilon,True)
  return orig_img_copy, approx

### This function takes the sudoku grid from the previous function
### and calcuulates the coordinates of the 4 corners of the grid 
def find_contour_points(approx):
  req_contour = approx.reshape((4,2))
  sum = req_contour.sum(axis = 1)
  diff = np.diff(req_contour, axis = 1)
  square = np.zeros((4,2), dtype = int)
  ### Top left and bottom right points will be the minimum and maximum of 
  ### the sum of the coordinates respectively
  square[0], square[2] = req_contour[np.argmin(sum)], req_contour[np.argmax(sum)]
  ### Top right and bottom left points will be the minimum and maximum of 
  ### the difference of the coordinates respectively
  square[1], square[3] = req_contour[np.argmin(diff)], req_contour[np.argmax(diff)]
  tl, tr, br, bl = square
  width_top = np.sqrt(((tl[0] - tr[0])**2) + ((tl[1] - tr[1])**2))
  width_bottom = np.sqrt(((bl[0] - br[0])**2) + ((bl[1] - br[1])**2))
  max_w = np.max([width_top, width_bottom])
  height_left = np.sqrt(((tl[0] - bl[0])**2) + ((tl[1] - bl[1])**2))
  height_right = np.sqrt(((tr[0] - br[0])**2) + ((tr[1] - br[1])**2))
  max_h = np.max([height_left, height_right])
  fin_points = np.array([[0,0],
                      [max_w-1, 0],
                      [max_w-1, max_h-1],
                      [0, max_h-1]], dtype = 'float32')
  return square, fin_points, max_w, max_h

### This function takes the extracted sudoku grid and flips it so as to get 
### a top-down view of the puzzle
def top_down_view(square, fin_points, orig_img_copy, max_w, max_h):
  persp = cv2.getPerspectiveTransform(square.astype('float32'), fin_points.astype('float32'))
  straight_img = cv2.warpPerspective(orig_img_copy, persp, (int(max_w), int(max_h)))
  return straight_img, persp

### This function takes the top down view of the puzzle, removes the noise,
### applies thresholding so as to separate the foreground from the background
def process_img(straight_img):
  straight_img_2 = cv2.fastNlMeansDenoisingColored(straight_img.copy(), None,10,10,7,21)
  gray_ = cv2.cvtColor(straight_img_2, cv2.COLOR_BGR2GRAY)
  gaussian_blur_ = cv2.GaussianBlur(gray_, (5,5), 0)
  ret,thresh_ = cv2.threshold(gaussian_blur_,127,255,cv2.THRESH_BINARY)
  warp = cv2.bitwise_not(thresh_)
  return warp

### This function divides the grid in to 9x9 parts, loops over them,
### differentiates between grid lines and the numbers, and extracts the 
### numbers when it finds one. In case there is no number, it takes the 
### value of 0. A numpy array of the puzzle is then created.
### 2 types of offsets are added here. The puzzle cannot be equally divided
### into 9 parts so the offset helps in making up for the remainder. Along with
### that, the offset for gridlines also needs to be made especially considering
### how the lines are thicker every 3 gridlines. 
def extract_sudoku_numbers(warp, model):
  pixel_val = []
  zeros_ind = []
  empty_puzzle = np.zeros((9,9))
  warp = warp[3:warp.shape[0] - 3, 3: warp.shape[1]- 3]
  h,w = warp.shape[0]//9, warp.shape[1]//9
  offset_height = h // 10
  offset_h_c = offset_height
  for i in range(9):
    row_sudoku = []
    offset_width = w // 10
    offset_height += (((i%3 == 0) and (i > 0))) 
    for j in range(9):
      increment_w_3 = (((j%3 == 0) and (j > 0)) * 2)
      addn_inc_w = j//3
      offset_width += increment_w_3
      y0 = (h * i)+ (offset_height); y1 = ((h * (i+1)) - (offset_height)//2)
      x0 = (w * j) + (offset_width) + (addn_inc_w); x1 = (w * (j+1) -  (offset_width//2))
      int_img = warp[y0:y1, x0:x1]
      box_h, box_w = int_img.shape[:2]
      ### measure.label() helps us find connected pixels which will help find the number
      labels = measure.label(int_img, neighbors=4, background = 0)
      biggest_contour = []
      ### finding the center area of the grid and taking the avg pixel value
      ### so as to check if there is a number of a blank grid
      center_y0, center_y1 = math.floor(int_img.shape[0]*0.25), math.ceil(int_img.shape[0]*0.75)
      center_x0, center_x1 = math.floor(int_img.shape[1]*0.25), math.ceil(int_img.shape[1]*0.75)
      average_center = np.mean(int_img[center_y0:center_y1, center_x0:center_x1])
      if average_center > 40:
       int_img = cv2.bitwise_not(int_img)
       ### looping through all connected pixels and taking the ones that are 
       ### present in the middle of the square so as to avoid taking gridlines
       for label in np.unique(labels):
          if label != 0:
            labelmask = np.zeros(shape = labels.shape, dtype = 'uint8')
            labelmask[labels == label] = 255.
            average_c = np.mean(labelmask[center_y0:center_y1, center_x0:center_x1])
            if average_c > 40:
              cnts, hi = cv2.findContours(np.array(labelmask), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
              req_cont = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
              rectangle_ = cv2.boundingRect(req_cont)
              numx,numy,numw,numh = rectangle_
              buffer_w = 5
              buffer_h = 5
              ### Adding a little buffer around the number
              int_img = int_img[np.max([0, numy-buffer_h]):np.min([int_img.shape[0], numy+numh+buffer_h]), np.max([0, numx-buffer_w]):np.min([int_img.shape[1], numx+numw+buffer_w])]
              ### centering the number to the center of the image
              ### so as to help improve accuracy
              int_img = shift_according_to_center_of_mass(int_img)
              ### resizing the image to 28x28 as required by the CNN model
              resized = cv2.resize(int_img, (28,28)).reshape((28,28,1))
              resized = np.expand_dims(resized, axis = 0)
              pred = np.argmax(model.predict(resized)) + 1
              row_sudoku.append(pred)
              
      else:
        ### If blank square, append 0
        row_sudoku.append(0)
        zeros_ind.append([i,j])
    pixel_val.append(row_sudoku)
  return pixel_val, zeros_ind

### Code to actually solve the sudoku puzzle. If no solution, then a matrix
### of zeros is returned
def solve_sudoku(pixel_val):
  if np.array(pixel_val).shape == (9,9):
    solve(pixel_val)
    final_sol = print_board(pixel_val)
  else:
    final_sol = np.zeros((9,9))
  return final_sol

### This function takes the solved puzzle, finds the points which were 
### originally empty, takes the solution, and prints it on the image
def text_on_sudoku(straight_img, zeros_ind, final_sol):
  img_h, img_w = np.array(straight_img.shape[:2])//9
  offset_width = math.floor(img_w / 10)
  offset_height = math.floor(img_h / 10)
  offset_h_c = offset_height
  offset_w_c = offset_width
  for i in range(9):
    for j in range(9):
      if [i,j] in zeros_ind:
        offset_h_c = math.floor((offset_h_c) - (i * offset_h_c))
        offset_w_c = math.floor((offset_w_c) - (i * offset_w_c))
        num = str(final_sol[i,j])
        init_x, init_y= ((img_w *j) + math.floor(0.5*img_w), (img_h * i) + math.ceil(0.6 * img_h) + offset_h_c)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(straight_img, num, (init_x, init_y), font, 0.6, (0,0,255), 2, cv2.LINE_AA)
  return straight_img

### This code takes the solved sudoku image and rotates it back to its original
### form.
def inverse_perspective_transform(straight_img, persp, original_image):
  result_sudoku = cv2.warpPerspective(straight_img, persp, (original_image.shape[1], original_image.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
  final_sudoku_image = np.where(result_sudoku.sum(axis=-1,keepdims=True)!=0, result_sudoku, original_image)
  return final_sudoku_image

### Wrapper code that calls all the above functions for an end to end 
### implementation of solving the sudoku puzzle. If no solution is found,
### the original image is returned.
def sudoku_solve(original_image):
  orig_image_copy, approx = find_contour(original_image)
  square, fin_points, max_w, max_h = find_contour_points(approx)
  straight_img, persp = top_down_view(square, fin_points, orig_image_copy, max_w, max_h)
  warp = process_img(straight_img)
  pixel_val, zeros_ind = extract_sudoku_numbers(warp, model)
  final_sol = solve_sudoku(pixel_val)
  if 0 in final_sol:
    return original_image
  else: 
    straight_img = text_on_sudoku(straight_img, zeros_ind, final_sol)
    final_sudoku_image = inverse_perspective_transform(straight_img, persp, original_image)
    return final_sudoku_image


In [10]:
### Taking the input video, running the code on each frame and returning the 
### frame with the required values and writing to a .mov file

cap = cv2.VideoCapture('sudoku_full_vid.mov')
writer = imageio.get_writer('sudoku_output.mov')
while True:
    ret, frame = cap.read()
    if not ret:
      break
    else:
      image_append= sudoku_solve(frame)
      image_append = cv2.cvtColor(image_append, cv2.COLOR_BGR2RGB)
      writer.append_data(image_append)
      
cap.release()
cv2.destroyAllWindows()
writer.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=4, use connectivity=1
